## Name: Sadnan Saquif
## Handle: SSaquif

# Safe dict reading

define a function `safe_dict(d, k)` that takes in a python dict `d` and a key `k` and makes it safe to read even with keys that aren't in the dictionary. If you try to read from the dictionary with a bad key, it should return 0 instead.

```
d = {1 : 2, 3 : 4}
safe_dict(d, 1) -> 2
safe_dict(d, 'cat') -> 0
```

In [1]:
safe_dict = lambda d, key: d[key] if key in d else 0

d = {1 : 2, 3 : 4}

print(safe_dict(d, 1))
print(safe_dict(d, 'cat'))

2
0


# File Reading: Hamlet Exercises

Open `hamlet.txt` in the `data` folder

### 1. Mentionned Hamlet

How many times is hamlet mentioned in the book?

Use python and line iteration to count it up

In [2]:
def hamlets_in_file(file):
    # Using with will close the file once it reaches EOF
    with open(file, 'r') as buffer:
        data = buffer.read()
    
    # Checking if file is closed    
    print(buffer.closed)
    hamlet_count = data.lower().count("hamlet")
    return hamlet_count

hamlets_in_file('./data/hamlet.txt')

True


474

### 2. File Reading as a .py program

Make a python file that defines a function that counts the number of times hamlet is mentionned using the code in the previous exercise.

Then import it in your notebook and call it here.

In [3]:
# Using file.write()
def create_py_file(file):
    # 'w' overwites existing content 
    with open(file, 'w') as buffer:
        buffer.write(
"""
def hamlets_in_file(file):
    with open(file, 'r') as buffer:
        data = buffer.read()

    return data.lower().count("hamlet")
"""
        )

# dont want to run again since the file exists
create_py_file('./hamlet_count.py')

In [4]:
from hamlet_count import hamlets_in_file

hamlets_in_file('./data/hamlet.txt')

474

### 3. Unique words in hamlet

Write a program that counts the unique words in hamlet.

In [5]:
# l1 = [1,2,3,4,5,6,1]
# s1 = len(set(l1))
# print(s1)
import re

def unique_words():
    with open('./data/hamlet.txt','r') as buffer:
        data = buffer.read()

    #splitting on space, comma, period, question mark and newline
    words = re.split('[ ,.?\n]', data.lower())
    # print(words)
    
    unique_words = len(set(words))
    return unique_words

unique_words()

5378

# File Reading 2: A Python library.

In the `data` folder, you will find a folder called `csrgraph` which is a python library.

### 1. File count

Count the `py` files in the library using the `os` package

In [6]:
import os

def pyfiles(directory):
    file_list = os.listdir(directory)
    py_files = []
    count = 0
    for file in file_list:
        if file.endswith('.py'):
            py_files.append(file)
    return py_files

file_list = pyfiles('./data/csrgraph/')
print(len(file_list))

8


### 2. For the following packages, count the number of files that import them:

- pandas 

- numpy

- numba

In [9]:
# Regex Testing
module = "numpy"
inp_1 = 'import   numpy\n'

inp_2 = 'import numpy as n_p \n'

inp_3 = 'from numpy import number \n'

# [ ]+ = 1 or more space
# [ ]* = 0 or more space
pattern_1 = f"(import)[ ]+({module})[ ]*\n"
pattern_2 = f"(import)[ ]+({module} as)[ ]+[a-zA-Z0-9_]+[ ]*\n"
pattern_3 = f"(from)[ ]+({module} import)[ ]+[a-zA-Z0-9_*]+[ ]*\n"

print(f"Pattern 1 {re.search(pattern_1,inp_1)}")
print(f"Pattern 2 {re.search(pattern_2,inp_2)}")
print(f"Pattern 3 {re.search(pattern_3,inp_3)}")

# print(pattern_1)
# print(inp1)

Pattern 1 <re.Match object; span=(0, 15), match='import   numpy\n'>
Pattern 2 <re.Match object; span=(0, 21), match='import numpy as n_p \n'>
Pattern 3 <re.Match object; span=(0, 26), match='from numpy import number \n'>


In [10]:
def used_in_files(directory, module):
    # Just check the python files in directory
    py_files = pyfiles(directory)
    
    used = 0
        
    for file in py_files:
        
        pattern_1 = f"(import)[ ]+({module})[ ]*\n"
        pattern_2 = f"(import)[ ]+({module} as)[ ]+[a-zA-Z0-9_]+[ ]*\n"
        pattern_3 = f"(from)[ ]+({module} import)[ ]+[a-zA-Z0-9_*]+[ ]*\n"
        
        with open(f"{directory}/{file}", 'r') as buffer:
            data = buffer.read()
            re_1 = re.search(pattern_1, data)
            re_2 = re.search(pattern_2, data)
            re_3 = re.search(pattern_3, data)

            if  re_1 or re_2 or re_3:
                used += 1
                
    return used
    
print(f"pandas: {used_in_files('./data/csrgraph/', 'pandas')}")
print(f"numpy: {used_in_files('./data/csrgraph/', 'numpy')}")
print(f"numba: {used_in_files('./data/csrgraph/', 'numba')}")

pandas: 4
numpy: 6
numba: 6


# First NLP Program: IDF

Given a list of words, the the inverse document frequency (IDF) is a basic statistic of the amount of information of each word in the text.

The IDF formulat is:

$$IDF(w) = ln(\dfrac{N}{1 + n(w)})$$

Where:

- $w$ is the token (unique word),
- $n(w)$ is the number of documents that $w$ occurs in,
- $N$ is the total number of documents

Write a function, `idf(docs)` that takes in a list of lists of words and returns a dictionary  `word -> idf score`

Example:

```
IDF([['interview', 'questions'], ['interview', 'answers']]) -> {'questions': 0.0, 
                                                                'interview': -0.4, 
                                                                'answers': 0.0}


```

# Stretch Goal: TF-IDF on Hamlet

The TF-IDF score is a commonly used statistic for the importance of words. Its $\frac{TF}{IDF}$ where TF is the "term frequency" (eg. how often the words happens in the document).

Calculate the TF-IDF dictionary on the Hamlet book.

What's the TF-IDF of "Hamlet"?

What's the word with the highest TF-IDF in the book?

# Stretch Goal: Speaker count

Use a regular expression and looping over the `hamlet.txt` file to build a dictionary `character_name -> # times speaking`.

Who speaks the most often? Who speaks the least often?